In [87]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction

import json
import requests
from pprint import pprint

In [88]:
# INPUT DATA
credentials = {
    'email': 'josepedrodsf@gmail.com',
    'password': 'conacona'
}
##antivirus = '2defaa98-8136-4382-b72b-2753d9697936'  # 'antivirus'
##cyber_security = '6928948d-051e-43f9-b5c7-012a9c40dc28'  # ' "cyber security" '
##football = '2734e9bd-5038-41be-8641-48ccd940670a'  # 'football'
##patches = 'bf9ad90d-3b91-401b-b3e3-d7040875e299'  # 'patches'
##hacking = '664a25f9-80b4-4cd3-94e7-76d7850cb5d8'  # 'hacking'
##cyber_attack = 'fe1dcdf9-8216-4215-aabe-0204fe6c30c8'  # '[cyber <8> "attack"]'
##cyber_regulation = 'f55b42f3-17cc-4262-aa0e-ff3316898e71'  # '[cyber <8> "regulation"]'
##cyber_future = '55f7018e-9372-4b44-9354-5fb33e91315a'  # '[cyber <8> "future"]'
##hack = '80fb257c-e8b2-49da-882c-155de3dc509f'  # 'hack'
##
##filter_cluster = [antivirus,cyber_security,football,patches,hacking,cyber_attack,cyber_regulation,cyber_future,hack]

example_filter_id='740f2770-e3f4-4d5d-becb-cd275d47bdef'




# /INPUT DATA




# GATHER ACCESS TOKEN
response = requests.post('https://api.owlin.com/v1/tokens', data=credentials)
token = json.loads(response.text)['token']
#print("The access token is {}".format(token))
authorization = {
    'authorization': token
}


# GATHER PROJECT ID (assuming there is a single one)
response = requests.get('https://api.owlin.com/v1/projects', headers=authorization)
projects = json.loads(response.text)
project_id = projects[0]['project_id']
#print("The project id is {}".format(project_id))


# REQUEST ARTICLES
example_query_parameters = {
    'published_at_lte': '1492099954',
    #'dedupe': 'disabled' # doesn't seem to be working, fixing it!
}



#for example_filter_id in filter_cluster:

filters_url = 'https://api.owlin.com/v1/projects/kpmg-hackdelft2/filters/{filter_id}/preview/stories?slicing_lte=5000&slicing_gte=100&dedupe=representatives_only'.format(project_id=project_id, filter_id=example_filter_id)

##filters_url = 'https://api.owlin.com/v1/projects/kpmg-hackdelft2/filters/{filter_id}\
##/preview/stories?slicing_lte=0&slicing_gte=250&dedupe=representatives_only'.format(project_id=project_id, filter_id=example_filter_id)
response = requests.get(filters_url, headers=authorization, params=example_query_parameters)

#filters_url = 'https://data.owlin.com/versions/api1:abbc30:1494701029/streams/group:hduuhqipwovpnuqwsygl0ut0f0gcyogr/articles?offset=0&max=1000&date=1494703511'

#response = requests.get(filters_url)
articlesList = json.loads(response.text)


In [89]:
articleAbstracts = []
for article in articlesList:
    articleAbstracts.append(article['text'])
    

In [90]:
articleTitles = []
for title in articlesList:
    articleTitles.append(article['title'])


In [91]:
print(str(len(articleTitles)) + 'titles')
print(str(len(articleAbstracts)) + 'abstracts')

3545titles
3545abstracts


In [92]:
ranks = []
for i in range(0,len(articleTitles)):
    ranks.append(i)
print(str(len(ranks)) + 'ranks')

3545ranks


In [93]:
stopwords = nltk.corpus.stopwords.words('english')

#stopwords.remove('you')
#stopwords.remove('your')
#stopwords.remove('yours')
#stopwords.remove('yourselves')
#stopwords.remove('yourself')

stopwords.append('\'s')
stopwords.append('us')

In [94]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [95]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

totalvocab_stemmed = []
totalvocab_tokenized = []
for i in articleAbstracts:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)    

In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.55, max_features=100000000, min_df=0.15, stop_words=stopwords, use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,4))
    
%time tfidf_matrix = tfidf_vectorizer.fit_transform(articleAbstracts)

print(tfidf_matrix.shape)

Wall time: 35.8 s
(3545, 20)


In [121]:
terms = tfidf_vectorizer.get_feature_names()

In [122]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [123]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [124]:
from sklearn.externals import joblib
joblib.dump(km, 'doc_cluster.pkl')


['doc_cluster.pkl']

In [125]:
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [126]:
import pandas as pd

articlesFrame = { 'title': articleTitles, 'rank': ranks, 'abstract': articleAbstracts, 'cluster': clusters }

frame = pd.DataFrame(articlesFrame, index = [clusters] , columns = ['rank', 'title', 'cluster'])

In [127]:
frame['cluster'].value_counts()

0    915
3    914
2    689
1    560
4    467
Name: cluster, dtype: int64

In [128]:
grouped = frame['rank'].groupby(frame['cluster'])

grouped.mean()

cluster
0    1804.538798
1    1711.694643
2    1692.489115
3    1791.101751
4    1860.483940
Name: rank, dtype: float64

In [ ]:
from __future__ import print_function

print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :3]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print()
    print()
    print("Cluster %d rank:" % i, end='')
    for rank in frame.ix[i]['rank'].values.tolist():
        print(' %s,' % rank, end='')
    print()
    print()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS

MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]
print()
print()

In [ ]:
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e', 5: '#585858', 6: '#FFFF00'}

#set up cluster names using a dict        
cluster_names = {0: 'cyber, attacks, threats', 
                 1: 'syria, russia, chemical', 
                 2: 'hospitals, hit, countries', 
                 3: 'market, global, reported', 
                 4: 'trump, north, president',
                 5: 'you, your, get',
                 6: 'said, attacks, states'}

In [ ]:
%matplotlib inline 

#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=ranks)) 

#group by cluster
groups = df.groupby('label')


# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
#for i in range(len(df)):
    #ax.text(df.ix[i]['x'], df.ix[i]['y'], df.ix[i]['title'], size=8)  

    
    
plt.show() #show the plot

#uncomment the below to save the plot if need be
#plt.savefig('clusters_small_noaxes.png', dpi=200)